# Новогодний скилл для Яндекс.Алиса с помощью DeepPavlov

Всем привет!
Сегодня бдуем писать бот для Яндекс.Алиса с использованием библиотеки [`deeppavlov`](https://github.com/deepmipt/DeepPavlov).
Наш бот будет уметь здороваться, прощаться и отвечать на вопросы по списку.
Приступим!

### Требования
Для реализации данного проекта вам потребуется виртуальная машина с выделеным IP адрессом. Работать надо в виртуальном окружении pyhton3.6 (`source activate py36`). Библиотеку можно установить через терминал командой:

`pip install deeppavlov`

# Теория

Две основные концепции в `deeppavlov` это `Skill` (не то же самое, что скилл для Алисы) и `Agent`.

### `Skill`
Расмсотрим самый базовый пример `Skill`'а. На вход он принимает сообщение пользователя `utterance` и возвращает ответ `response` и `confidence` от 0 до 1 - уверенность, в том, что ответ подходит.

![title](img/skill_example.png)

Кроме этого `Skill` на вход также принимает историю сообщений (`history`) и некое внутреннее состояние (`state`), смысл которого мы определяем сами. Внутренне состояние возвращается тем же `Skill`'ом на ряду с `responce` и `confidence`. Это сделано для реализации более сложного поведения бота, но в нашем случае эти поля пустые и никак не используются.

### `Agent`
Теперь рассмотрим базовый пример `Agent`'а. `Agent` по сути представляет собой набор `Skill`'ов и механизм выбора наиболее подходящего из них: `skills_selector`. На вход `Agent` принимает сообщение пользователя `utterance` и передает его каждому `Skill`'у. Далее на основе выдачи всех `Skill`'ов `skills_selector` выбирает самый подходящий и возвращает пользователю его ответ. В данном примере просто выбирается `Skill`, вернувший самую большую уверенность (`confidence`).

![title](img/agent_example.png)

Библиотка `deeppavlov` предоставляет набор разных `Skill`'ов и `Agent`'ов. Также она предоставляет набор NLP (Natural Language Processing) инстрементов, которые дают возможность создавать свои `Skill`'ы.

# Реализация

Сперва создадим самый простой `Skill`, который будет отвечать за приветствие - `PatternMatchingSkill`. При его создании мы задаем `patterns`. Чем больше совпадений между сообщением пользователя (`utterance`) и `patterns`, тем больше уверенность (`confidence`), которую вернет `Skill`. Ответ выбирается случайным образом из заданных нами `responces`.

In [3]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill

hello = PatternMatchingSkill(responses=['Привет!', 'Приветик', 'Здравствуйте', 'Привет, с наступающим Новым Годом!'],
	patterns=['Привет', 'Здравствуйте', 'Добрый день'])

Таким же образом образом обучим бота прощаться.

In [4]:
bye = PatternMatchingSkill(responses=['Пока!', 'До свидания, надеюсь смог вам помочь', 'C наступающим Новым Годом!'],
	patterns=['До свидания', 'Пока', 'Спасибо за помощь'])

Если в `PatternMatchingSkill` не указать `patterns`, то он всегда будет возвращать уверенность `0.5`. Это можно использовать для создания скилла `fallback`, который отвечает на непонятные запросы: если все созданные нами `Skill`'ы возвращают маленький `confidense` (пользователь написал что-то неожиданное или непонятное), то `Agent` выберет ответ `fallback`, как самый подходящий.

In [5]:
fallback = PatternMatchingSkill(responses=['Я не понял, но могу попробовать ответить на другой вопрос',
	'Я не понял, задайте другой вопрос'])

Впереди самое интересное: научить нашего бота отвечать на вопросы. Создадим табличку `'faq.csv'` и поместим ее в папку `data`. Заполним табличку нашими вопросами и ответами и окинем ее взглядом.

In [6]:
import pandas as pd
faq_table = pd.read_csv('data/faq_doctor.csv')
faq_table

,Question,Answer
0,Болит живот,Скушайте таблетку от живота
1,Боль в животе,Скушайте таблетку от живота
2,Колит желудок,Скушайте таблетку от живота
3,Режет желудок,Скушайте таблетку от живота
4,Острая боль внизу живота,Скушайте таблетку от живота
5,Болит голова,Скушайте таблетку от головы
6,Очень сильно болит голова,Скушайте таблетку от головы
7,Сильная головная боль,Скушайте таблетку от головы
8,У меня мигрень,Скушайте таблетку от головы
9,Хочу кушать,Ну так покушай


Обратите внимание, что для одних и тех же ответов указано несколько разных вопросов. Это сделано для того, чтобы бот лучше распознавал различные формулировки. Еще один важный момент: в табличке обязательно нужно назвать стоблец с вопросами `Question`, а с ответами `Answer`, иначе алгоритм выдаст ошибку.

Еще одной важной концепцией в `deeppavlov` яляются `config`'и, которые представляют собой `.json` файлы. Они позволяют относительно кратко описывать NLP модели. В библиотеке уже есть большое количество описаных моделей, в том числе и для нашей задачи. Возьмем такую модель и исправим у нее ссылку на таблицу, по которой модель обучается и, собственно, обучим ее.

In [7]:
from deeppavlov.core.common.file import read_json
from deeppavlov.core.common.file import find_config
from deeppavlov import train_model

model_config = read_json(find_config('tfidf_autofaq'))
model_config['dataset_reader']['data_url'] = 'data/faq_doctor.csv'
faq = train_model(model_config)

2018-12-17 20:08:35.146 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /usr/local/miniconda3/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-12-17 20:08:35.215 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
2018-12-17 20:08:35.219 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 201: Loading model sklearn.feature_extraction.text:TfidfVectorizer from /Users/nv/.deeppavlov/models/vectorizer/tfidf_vectorizer_ruwiki.pkl
2018-12-17 20:08:35.221 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 208: Model sklearn.feature_extraction.textTfidfVectorizer loaded  with parameters
2018-12-17 20:08:35.222 WARNING in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 214: Fitting of loaded model can not be continued. Model can be fitted from scratch.If one needs to contin

Мы уже на финишной прямой, остлалось собрать все `Skill`'ы вместе при помощи `Agent`'а.

In [8]:
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector

agent = DefaultAgent([hello, bye, faq, fallback], skills_selector=HighestConfidenceSelector())

Проверим код на работоспобность. Агент может принимать на вход сразу несколько вопросов:

In [9]:
print(agent(['Привет', 'У меня болит живот', 'Пока', 'бла бла бла']))

['Привет, с наступающим Новым Годом!', 'Скушайте таблетку от живота', 'Пока!', 'Я не понял, задайте другой вопрос']


/usr/local/miniconda3/lib/python3.6/site-packages/deeppavlov/models/classifiers/cos_sim_classifier.py:81: RuntimeWarning: invalid value encountered in true_divide
  labels_scores = labels_scores/labels_scores.sum(axis=1, keepdims=True)


Пора запускать код в бой. При помощи простой утилиты можно заставить работать нашего агента в режиме web-hook, к которому будут подключаться сервера Яндекса. Но прежде нам надо создать ssl ключ и сертификат. Для этого в терминале выполним команду, где вместо `<MACHINE_IP_ADDRESS>` надо указать внешний IP адрес вашей виртуальный машины.

`openssl req -new -newkey rsa:4096 -days 365 -nodes -x509 -subj "/CN=<MACHINE_IP_ADDRESS>" -keyout my.key -out my.crt`

In [ ]:
from utils.alice import start_agent_server

start_agent_server(agent, host='0.0.0.0', port=5000, endpoint='/faq', ssl_key='my.key', ssl_cert='my.crt')

 * Serving Flask app "utils.alice.alice" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2018-12-16 21:07:06.819 INFO in 'werkzeug'['_internal'] at line 88:  * Running on https://0.0.0.0:5000/ (Press CTRL+C to quit)
/opt/conda/envs/py36/lib/python3.6/site-packages/deeppavlov/models/classifiers/cos_sim_classifier.py:63: RuntimeWarning: invalid value encountered in true_divide
  cos_similarities = np.array(q_vects.dot(self.x_train_features.T).todense())/norm
/opt/conda/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
2018-12-16 21:07:19.850 INFO in 'werkzeug'['_internal'] at line 88: 93.158.158.74 - - [16/Dec/2018 21:07:19] "POST /faq HTTP/1.1" 200 -
2018-12-16 21:07:26.578 INFO in 'werkzeug'['_internal'] at line 88: 93.158.158.74 - - [16/Dec/2018 21:07:26] "POST /faq HTTP/1.1" 200 -


Теперь в консоле Яндекс.Диалоги укажем web-hook `<MACHINE_IP_ADDRESS>:5000/faq` и во вкладке тестирование можем проверить нашего бота.

![title](img/dialog_example.png)

# Заключение
Итак, не более 10 содержательных строк потребовалось нам, чтобы запустить на Яндекс.Алиса бота с весьма полезным функционалом. Будем очень рады услышать ваш фидбек по DeepPavlov на нашей странице в [GitHub](https://github.com/deepmipt/DeepPavlov).